In [ ]:
from datasets import load_dataset, Dataset
from polyglot.detect import Detector
from polyglot.detect.base import UnknownLanguage
from tqdm import tqdm
import langcodes

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [ ]:
def pop_mc4(ds, lang, size=1000000):
    docs = {'text':[]}
    for example in tqdm(ds['train'], total=size, desc='pop dataset'):
        text = example['text'].replace('\n', ' ')
        docs['text'].append(text)
        if len(docs['text']) == size:
            break
    return docs

In [ ]:
#list_langs = ['german', 'greek', 'spanish', 'hindi', 'russian', 'thai', 'turkish', 'vietnamese', 'chinese', 'italian', 'french', 'portuguese', 'indonesian', 'japanese']
list_langs = ['korean']
path = "/path/to/mc4"
max_size = 3010000

for lang in list_langs:
    tag = langcodes.find(lang).to_tag()
    ds_iter = load_dataset("allenai/c4", tag, streaming=True)
    docs = pop_mc4(ds_iter, lang=lang, size=max_size)
    ds = Dataset.from_dict(docs)
    docs = None
    del docs
    ds = ds.train_test_split(test_size=5000)
    ds.save_to_disk(f"{path}/{lang}")
    print(f"finish pop dataset for {lang}")
    ds = None
    del ds

In [1]:
import torch
import random
import datasets
from datasets import load_dataset, Dataset
from datasets import concatenate_datasets
from datasets import disable_caching
disable_caching()

#langs = ['arabic', 'bangla', 'chinese', 'english', 'finnish', 'french', 'german', 'hindi', 'indonesian', 'japanese', 'korean', 'persian', 'russian', 'spanish', 'telugu', 'thai']
#abbvs = 'ar bn zh en fi fr de hi id ja ko fa ru es te th'.split()

langs = ['telugu', 'thai']
abbvs = 'te th'.split()

In [2]:
for lang, abbv in zip(langs, abbvs):
    print(lang, abbv)
    path = f"/path/to/miracl/{abbv}/collection.tsv"
    mir_ds = load_dataset("csv", delimiter="\t", header=None, names=['pid', 'text'], usecols=[0, 1], data_files=path)
    mir_ds = mir_ds['train']
    mir_ds = mir_ds.remove_columns("pid")
    mir_len = len(mir_ds)
    if mir_len >= 3005000:
        mir_ds = mir_ds.shuffle(seed=42)
        ds = mir_ds.select(range(3005000))
    else:
        res = 3005000 - mir_len
        mc4_ds = datasets.load_from_disk(f"/path/to/mc4/{lang}/train")
        mc4_len = len(mc4_ds)
        indexes = random.sample(range(mc4_len), min(res, mc4_len))
        mc4_ds = mc4_ds.select(indexes)
        ds = concatenate_datasets([mir_ds, mc4_ds])

    ds = ds.train_test_split(test_size=5000)
    ds.save_to_disk(f"/path/to/mix4/{lang}")
    print(f"finish pop dataset for {lang}")
    ds = None
    mir_ds = None
    mc4_ds = None

telugu te


Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Saving the dataset (0/19 shards):   0%|          | 0/1696322 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

finish pop dataset for telugu
thai th


Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset from disk:   0%|          | 0/35 [00:00<?, ?it/s]

Saving the dataset (0/31 shards):   0%|          | 0/3000000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

finish pop dataset for thai


In [ ]:
mc4_len